In [1]:
import pandas as pd 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
import re
from collections import Counter
from nltk import ngrams
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df['target'].value_counts()
df['title'].value_counts()

rec.sport.hockey            600
soc.religion.christian      599
rec.motorcycles             598
rec.sport.baseball          597
sci.crypt                   595
rec.autos                   594
sci.med                     594
comp.windows.x              593
sci.space                   593
comp.os.ms-windows.misc     591
sci.electronics             591
comp.sys.ibm.pc.hardware    590
misc.forsale                585
comp.graphics               584
comp.sys.mac.hardware       578
talk.politics.mideast       564
talk.politics.guns          546
alt.atheism                 480
talk.politics.misc          465
talk.religion.misc          377
Name: title, dtype: int64

In [22]:
# df = df.replace(r'\n',' ', regex=True) 
# df = df.replace(r'\t',' ', regex=True) 
# df = df.replace(r"[^a-zA-Z]+",' ', regex=True) 
# df['text'] = df['text'].str.lower()
# df

,text,target
0,i was wondering if anyone out there could enli...,7
1,i recently posted an article asking what kind ...,7
2,it depends on your priorities a lot of people...,7
3,an excellent automatic can be found in the sub...,7
4,ford and his automobile i need information on...,7
...,...,...
11309,secrecy in clipper chip the serial number of t...,11
11310,hi i am interested in the source of feal encry...,11
11311,the actual algorithm is classified however the...,11
11312,this appears to be generic calling upon the n...,11


In [31]:
from sklearn.datasets import fetch_20newsgroups


categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(
    subset='train', categories=categories,
    shuffle=True, random_state=42)
df = pd.DataFrame(list(zip(twenty_train.data, twenty_train.target)), columns=['text', 'category'])
df.to_csv("clean.csv", index = False)
df

,text,category
0,From: sd345@city.ac.uk (Michael Collier)\nSubj...,1
1,From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\...,1
2,From: djohnson@cs.ucsd.edu (Darin Johnson)\nSu...,3
3,From: s0612596@let.rug.nl (M.M. Zwart)\nSubjec...,3
4,From: stanly@grok11.columbiasc.ncr.com (stanly...,3
...,...,...
2252,From: roos@Operoni.Helsinki.FI (Christophe Roo...,2
2253,From: mhollowa@ic.sunysb.edu (Michael Holloway...,2
2254,From: sasghm@theseus.unx.sas.com (Gary Merrill...,2
2255,From: Dan Wallach <dwallach@cs.berkeley.edu>\n...,2


In [32]:
df.replace("", float("NaN"), inplace=True)
df.dropna(inplace=True)
df.isnull().values.any()
df

,text,category
0,From: sd345@city.ac.uk (Michael Collier)\nSubj...,1
1,From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\...,1
2,From: djohnson@cs.ucsd.edu (Darin Johnson)\nSu...,3
3,From: s0612596@let.rug.nl (M.M. Zwart)\nSubjec...,3
4,From: stanly@grok11.columbiasc.ncr.com (stanly...,3
...,...,...
2252,From: roos@Operoni.Helsinki.FI (Christophe Roo...,2
2253,From: mhollowa@ic.sunysb.edu (Michael Holloway...,2
2254,From: sasghm@theseus.unx.sas.com (Gary Merrill...,2
2255,From: Dan Wallach <dwallach@cs.berkeley.edu>\n...,2


In [13]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,target
0,weitek address phone number like get informati...,1
1,certainly use whenever tiff usually works well...,1
2,hello looking add voice input capability user ...,1
3,australian pattern recognition society nd call...,1
4,hi noel made attempts write converter reads ad...,1
...,...,...
1650,excerpts netnews alt atheism apr thoughts chri...,0
1651,right humans gone somewhat beyond though perha...,0
1652,miss post topic quote indonesian handbook fred...,0
1653,archive name atheism resources alt atheism arc...,0


In [28]:
 # Creating train-test Split
from sklearn.model_selection import train_test_split

categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']

df_train = pd.DataFrame(list(zip(df.text, df.target)), columns=['text', 'category'])
X = df[['text']]
y = df[['target']]

df_train


In [27]:
df_train.to_csv("clean.csv", index = False)
df_train

,text,category
0,From: sd345@city.ac.uk (Michael Collier)\nSubj...,1
1,From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\...,1
2,From: djohnson@cs.ucsd.edu (Darin Johnson)\nSu...,3
3,From: s0612596@let.rug.nl (M.M. Zwart)\nSubjec...,3
4,From: stanly@grok11.columbiasc.ncr.com (stanly...,3
...,...,...
2252,From: roos@Operoni.Helsinki.FI (Christophe Roo...,2
2253,From: mhollowa@ic.sunysb.edu (Michael Holloway...,2
2254,From: sasghm@theseus.unx.sas.com (Gary Merrill...,2
2255,From: Dan Wallach <dwallach@cs.berkeley.edu>\n...,2


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

vec = CountVectorizer()
clf = LogisticRegression()
pipe = make_pipeline(vec, clf)
pipe.fit(X_train.text, y_train.target)

In [16]:
clf.classes_ 

array([0, 1, 2], dtype=int64)

In [17]:
import eli5
feature_imp = eli5.show_weights(clf, vec=vec, top=100)
feature_imp

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from wordcloud import WordCloud
text = " ".join(cat for cat in df.word)
word_cloud = WordCloud(collocations = False, background_color = 'white').generate(text)
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud.png')
plt.show()